# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
import os
import pandas as pd

In [2]:
DATA_FOLDER = '../ADA2017-Tutorials/02 - Intro to Pandas/Data'
EBOLA_FOLDER = os.path.join(DATA_FOLDER, 'ebola')

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

### Solution Task 1

#### Assumptions
- The dicionary `schema_and_assumptions` has for each country:
 1. `col_date` is the name of the column containing the date
 2. `col_desc` the name of the columns containing the description
 3. `desc_new_cases` contains the strings that will be considered as new_cases
 4. `desc_death` contains the strings that will be considered as death
 
 This dicionary will be used to filter each file in the function `get_new_and_death`


- The date will be extracted from the file's name because they don't have a standard inside the files, so it would be painful and risky and map all formats

- Even for the same country, there are different columns for the files, so the first step is look for all columns available in all files and then for each file it will be selected just the columns that is available (done in function `read_csv_group_concat`)

- The function `standardize_date_in_file_name` will standardize the dates

- Some columns are not read as numeric type for `sl_data` files, this causes high numbers that looks like outliers but it is just an error when the files are loaded

- There missing observations for some dates and countries (specially for Guinea). The missing will not be considered or treated, it means, the mean will be calculated without those observations. This is better than fill with 0's or doing some kind of educated guess.

#### Conclusions and Final Comments
- Accuracy for `guinea_data` is lower compared with other groups because it just have 22 observed days which is 5x less than the other groups


- For `guinea_data` and `sl_data` the number of deaths is much higher than `new_cases` and this might be because:
  1. We are not filtering the rights descriptions in the `schema_and_assumptions` for `desc_new_cases` and `desc_death`. This can be fixed with a proper metadada, which is hard to have (rs).
  2. It is easier collect deaths than new_cases, so most of the registred cases might be not recorded
  3. The period of time that we have observations is biased, might be the end of an outbreak so the number of deaths will be higher than the number os new cases

In [3]:
schema_and_assumptions = {'liberia_data': {'col_date': 'Date',
                                           'col_desc': 'Variable',
                                           'desc_new_cases': ['New case/s (confirmed)',
                                                              'New Case/s (Probable)',
                                                              'New Case/s (Suspected)'],
                                           'desc_death': ['Total death/s in confirmed cases',
                                                          'Total death/s in confirmed cases',
                                                          'Total death/s in suspected cases']
                                           },

                          'sl_data': {'col_date': 'date',
                                      'col_desc': 'variable',
                                      'desc_new_cases': ['new_confirmed',
                                                         'new_probable',
                                                         'new_suspected'],
                                      'desc_death': ['death_confirmed',
                                                     'death_probable',
                                                     'death_suspected']
                                      },

                          'guinea_data': {'col_date': 'Date',
                                          'col_desc': 'Description',
                                          'desc_new_cases': ['New cases of confirmed',
                                                             'New cases of probables',
                                                             'New cases of suspects'],
                                          'desc_death': ['Total deaths of confirmed',
                                                         'Total deaths of probables',
                                                         'Total deaths of suspects']
                                          }
                          }

In [4]:
def standardize_date_in_file_name(messed_date):
    '''
    Receives something like 2014-12-09-v920 and returns 2014-12-09
    Hacked and dirty
    '''
    return messed_date[0:10]

In [5]:
def get_new_and_death(df, 
                      country,
                      date, 
                      col_desc, 
                      cols_to_sum, 
                      desc_new_cases, 
                      desc_death):
    '''
    1. Drop columns that are not in `{date, col_desc, cols_to_sum}`
    2. Drop lines that are not in `{desc_new_cases, desc_death}`
    3. Convert `cols_to_sum` in numeric (someone them are not, them it causes fake-outliers)
    4. For each line it will sum all the columns in cols_to_sum
    5. Aggregate sums in the `{new_cases, death}`
    6. Standardize columns names
    
    :returns a dataframe with 1 row and the following columns `{date, country, type {deaths, new_cases}, total}`
    '''
    
    df = df[[col_desc] + cols_to_sum]
    df = df[df[col_desc].apply(lambda x: x in desc_new_cases + desc_death)]
    df[cols_to_sum] = df[cols_to_sum].fillna(0).apply(pd.to_numeric)

    df['total'] = df[cols_to_sum].sum(axis=1)
    df['type'] = df[col_desc].apply(lambda x: 'new_cases' if x in desc_new_cases else 'deaths')
    
    df['date'] = date
    agg_df = (df
              .groupby(['date', 'type'])
              .sum()[['total']].reset_index())
    agg_df['country'] = country
    
    return agg_df[['date', 'country', 'type', 'total']]

In [6]:
def read_csv_group_concat(root_path, schemas):
    '''
    1. For each country it scan all columns of all files
    2. Colect all possible columns and filter that ones that don't make sense like: Unamed, Totals, `col_date` and `col_desc`
    3. For each file, it aggregates using the function `get_new_and_death`
    4. Concat all the results
    
    :returns a dataframe with the following columns `{date, country, type {deaths, new_cases}, total}`
    '''
    
    agg_counts = pd.DataFrame()

    for country in schemas:
        schema = schemas[country]
        
        country_path = os.path.join(root_path, country)
        all_csvs = filter(lambda x: x.endswith('csv'), os.listdir(country_path))
        all_paths = list(map(lambda x: os.path.join(country_path, x), all_csvs))
        
        scan_all_cols = map(lambda p: list(pd.read_csv(p, nrows=0)), all_paths)
        all_possible_cols = list(filter(lambda x: not x.startswith('Unnamed') and 
                                                  not x.startswith('Totals') and
                                                  x!=schema['col_date'] and
                                                  x!=schema['col_desc'],
                                        list(set(sum(scan_all_cols, [])))))
    
        for file_path in all_paths:
            df = pd.read_csv(file_path)
            cols_to_sum = list(set(all_possible_cols).intersection(set(df.columns)))
            date = standardize_date_in_file_name(os.path.split(file_path)[-1])
            
            agg_df = get_new_and_death(df,
                                       country,
                                       date,
                                       schema['col_desc'],
                                       cols_to_sum,
                                       schema['desc_new_cases'],
                                       schema['desc_death'])
                
            agg_counts = pd.concat([agg_counts, agg_df], axis=0)
                
    agg_counts.date = pd.to_datetime(agg_counts.date)
    
    return agg_counts

In [36]:
agg_by_date = read_csv_group_concat(EBOLA_FOLDER, schema_and_assumptions)
mean_by_year = agg_by_date.groupby(['country', 'type']).agg(['size', 'mean']).unstack(level=-1)

In [37]:
print('Daily average for the observations between {} and {}'.format(min(agg_by_date['date']).date(), max(agg_by_date['date']).date()))
mean_by_year['total']

Daily average for the observations between 2014-06-16 and 2014-12-13


size                   mean            
type         deaths new_cases       deaths   new_cases
country                                               
guinea_data      22        22   570.363636   21.454545
liberia_data    100       100   716.750000  995.280000
sl_data         103       103  1735.864078  107.747573

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here